In [1]:
import argparse

parser = argparse.ArgumentParser(description='Options for the analyzer')
parser.add_argument('--name', metavar='n', action='store', type=str,help='short name of the physics process/Data sample', required=True)
parser.add_argument('--file', metavar='f', action='store', type=str,help='location of files',required=True)
parser.add_argument('--saveroot',action='store_true', help='save root file',default=False)


args = parser.parse_args()

sysName=args.name
sysfiles=args.file
saveRootFile=args.saveroot

In [2]:
import ROOT
ROOT.ROOT.EnableImplicitMT()
ROOT.gInterpreter.AddIncludePath("./include/");
ROOT.gInterpreter.ProcessLine('#include "Helper.h"')
from include.PyHelper import *

Welcome to JupyROOT 6.22/06


In [3]:
ROOT.gStyle.SetOptStat(0)

In [4]:
goodelectrons=ObjectMask()
goodelectrons.addcut("(Electron_pt > 10)")
goodelectrons.addcut("((Electron_sieie<0.011)&&(abs(Electron_eta)<=1.479))|| ((Electron_sieie<0.030)&&(abs(Electron_eta)>1.479))")
goodelectrons.addcut("abs(Electron_dxy)< 0.05")
goodelectrons.addcut("Electron_mvaTTH > 0.8")
goodelectrons.addcut("Electron_hoe<0.1")
goodelectrons.addcut("Electron_eInvMinusPInv>-0.04")
goodelectrons.addcut("Electron_sip3d<8")
goodelectrons.addcut("Electron_convVeto")
goodelectrons.addcut("Electron_mvaFall17noIso_WPL")
goodelectrons.addcut("(Electron_pfRelIso03_all < (0.4*Electron_pt))")
goodelectrons.addcut("Electron_lostHits==0")
goodelectrons.addcut("(abs(Electron_eta)< 1.479)")

Ana= MyAnalyzer()
Ana.definition("GoodElectrons",goodelectrons.mask)
Ana.definition("NGoodElectrons","Sum(GoodElectrons)")
Ana.definition("MyElectron_pt","Take(Electron_pt[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_eta","Take(Electron_eta[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_phi","Take(Electron_phi[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_mass","Take(Electron_mass[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_charge","Take(Electron_charge[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_genPartIdx","Take(Electron_genPartIdx[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_jetIdx","Take(Electron_jetIdx[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))")
Ana.definition("MyElectron_pt1","MyElectron_pt[0]")
Ana.definition("MyElectron_eta1","MyElectron_eta[0]")
Ana.definition("MyElectron_phi1","MyElectron_phi[0]")
Ana.definition("MyElectron_mass1","MyElectron_mass[0]")
Ana.definition("MyElectron_charge1","MyElectron_charge[0]")
Ana.definition("MyElectron_genPartIdx1","MyElectron_genPartIdx[0]")
Ana.definition("MyElectron_jetIdx1","MyElectron_jetIdx[0]")
Ana.definition("MyElectron_motherpid1","GenPart_pdgId[GenPart_genPartIdxMother[MyElectron_genPartIdx1]]")
Ana.definition("MyElectron_deepjet1","Jet_btagDeepFlavB[MyElectron_jetIdx1]")
Ana.definition("MyElectron_pt2","MyElectron_pt[1]")
Ana.definition("MyElectron_eta2","MyElectron_eta[1]")
Ana.definition("MyElectron_phi2","MyElectron_phi[1]")
Ana.definition("MyElectron_mass2","MyElectron_mass[1]")
Ana.definition("MyElectron_charge2","MyElectron_charge[1]")
Ana.definition("MyElectron_genPartIdx2","MyElectron_genPartIdx[1]")
Ana.definition("MyElectron_jetIdx2","MyElectron_jetIdx[1]")
Ana.definition("MyElectron_motherpid2","GenPart_pdgId[GenPart_genPartIdxMother[MyElectron_genPartIdx2]]")
Ana.definition("MyElectron_deepjet2","Jet_btagDeepFlavB[MyElectron_jetIdx2]")
Ana.definition("Dielectron_mass","ComputeInvariantMass(MyElectron_pt, MyElectron_eta, MyElectron_phi, MyElectron_mass)")

Ana.alias("elpt","MyElectron_pt")
Ana.alias("eleta","MyElectron_eta")
Ana.alias("elphi","MyElectron_phi")
Ana.alias("elmass","MyElectron_mass")
Ana.alias("elq","MyElectron_charge")
Ana.alias("elpt1","MyElectron_pt1")
Ana.alias("eleta1","MyElectron_eta1")
Ana.alias("elphi1","MyElectron_phi1")
Ana.alias("elmass1","MyElectron_mass1")
Ana.alias("elq1","MyElectron_charge1")
Ana.alias("elpt2","MyElectron_pt2")
Ana.alias("eleta2","MyElectron_eta2")
Ana.alias("elphi2","MyElectron_phi2")
Ana.alias("elmass2","MyElectron_mass2")
Ana.alias("elq2","MyElectron_charge2")

Ana.sel("Sum(GoodElectrons) == 2","Two or more good electrons")
#Ana.sel("MyElectron_motherpid1==23 && MyElectron_motherpid2==23","Come from Z")
Ana.sel("(MyElectron_deepjet1<0.3033) && (MyElectron_deepjet2<0.3033)","DeepJet cuts on ele")
Ana.sel("(Dielectron_mass<121) && (Dielectron_mass>61)","Z mass cut")
Ana.sel("DeltaR(eleta1, eleta2, elphi1, elphi2)>0.4","DeltaR cut")
Ana.sel("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter","Quality1")
Ana.sel("Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter","Quality2")
Ana.sel("Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter","Quality3")


#file="root://xrootd-cms.infn.it///store/mc/RunIISummer16NanoAODv3/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/PUMoriond17_94X_mcRun2_asymptotic_v3_ext2-v1/100000/04652A9D-99C3-E811-9222-B4969109FDE8.root"
DYfiles='/eos/cms/store/group/phys_egamma/akapoor/ChargeMisID/newsamples2/*.root'
if hasattr(__builtins__,'__IPYTHON__'):
    Ana.process("DY",{'Files':DYfiles,'Type':'Bkg','Tree':'Events','Xsec':1})
else:
    Ana.process(str(sysName),{'Files':str(sysfiles),'Type':'Bkg','Tree':'Events','Xsec':1})
    #Ana.process(sys.srgv[1],{'Files':DYfiles,'Type':'Bkg','Tree':'Events','Xsec':1})

In [5]:
Ana.prepare()

Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
#for PD in PDs:
#    PD['RDF'].Display({"NGoodElectrons","elpt","MyElectron_pt"}).Print()

In [7]:
#hMeeSS = PDs[0]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("Dielectron_mass_SS", "Dielectron_mass;m_{ee} (GeV);Events", 10, 71, 111), "Dielectron_mass")
#hMee = PDs[0]['RDF'].Histo1D(("Dielectron_mass_all", "Dielectron_mass;m_{ee} (GeV);Events", 10, 71, 111), "Dielectron_mass")


#hemotherpid1 = PDs[0]['RDF'].Histo1D(("MyElectron_motherpid1", "MyElectron_motherpid1;MyElectron_motherpid1;Events", 50, -25, 25), "MyElectron_motherpid1")

#helpt1SS = PDs[0]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("elpt1_SS", "elpt1;elpt1 (GeV);Events", 5, 10, 100), "elpt1")
#helpt1 = PDs[0]['RDF'].Histo1D(("elpt1_all", "elpt1;elpt1 (GeV);Events", 5, 10, 100), "elpt1")

In [8]:
#savehist([hMeeSS.Clone(),hMee.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=True)

#savehist([hemotherpid1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

#savehistonlyratio([hMeeSS.Clone(),hMee.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

In [9]:
#savehist([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=True)
#savehist([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

#savehistonlyratio([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

In [10]:
#print(PDs[0]['RDF'].GetColumnNames())

In [11]:
#PDs[0]['RDF'].Display({"MyElectron_pt","MyElectron_motherpid1"}).Print()

DF=Ana.PD['DY']['RDF'].Filter("elq1>0","el is positive")

In [12]:
GlobalRate=DF.Filter("(elq1+elq2)!=0", "Two SS").Count().GetValue() / DF.Count().GetValue()

In [13]:
print(f"The global charge flip rate is {round(GlobalRate*100,3)}%")

The global charge flip rate is 0.469%


In [14]:
printCutFlow(DF)

Two or more good electrons: pass=206007     all=4621815    -- eff=4.46 % cumulative eff=4.46 %
DeepJet cuts on ele: pass=203821     all=206007     -- eff=98.94 % cumulative eff=4.41 %
Z mass cut: pass=196059     all=203821     -- eff=96.19 % cumulative eff=4.24 %
DeltaR cut: pass=195749     all=196059     -- eff=99.84 % cumulative eff=4.24 %
Quality1  : pass=195697     all=195749     -- eff=99.97 % cumulative eff=4.23 %
Quality3  : pass=195618     all=195697     -- eff=99.96 % cumulative eff=4.23 %
el is positive: pass=97522      all=195618     -- eff=49.85 % cumulative eff=2.11 %


In [15]:
if saveRootFile:
    #std::vector<string> save_cols ={"Electron_pt"}
    DF.Snapshot("Events","Snapshot.root",{"elpt1"})